In [5]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']


In [6]:

# --- 1. 物理参数和LLG方程定义 ---
alpha = 0.02
gamma = 1.7e7 / 1e9
H_ext = 200.0
H_deg = 8400.0
omega_ac_GHz = 32.0
omega_ac = omega_ac_GHz * 2 * np.pi
a_ac = 26.0
e_x = np.array([1.0, 0.0, 0.0])
e_z = np.array([0.0, 0.0, 1.0])

# a_j(t) 将在主函数中定义，因为它在计算LLE时是固定的

def llg_6d(t, y, get_aj_func):
    """
    求解6维系统的LLG方程，用于LLE计算。
    y = [m_ref, m_pert] 是一个6维向量。
    """
    m_ref = y[:3]
    m_pert = y[3:]
  
    # 确保矢量归一化 (对数值误差鲁棒)
    m_ref = m_ref / np.linalg.norm(m_ref)
    m_pert = m_pert / np.linalg.norm(m_pert)
  
    # 对两个轨迹使用完全相同的驱动信号 a_j(t)
    a_j_t = get_aj_func(t)
  
    # --- 计算参考轨迹的 d(m_ref)/dt ---
    H_eff_ref = H_ext * e_x - H_deg * m_ref[2] * e_z
    T_field_ref = -gamma * np.cross(m_ref, H_eff_ref)
    T_stt_ref = gamma * a_j_t * np.cross(m_ref, np.cross(m_ref, e_x))
    T_total_ref = T_field_ref + T_stt_ref
    dmdt_ref = (1 / (1 + alpha**2)) * (T_total_ref + alpha * np.cross(m_ref, T_total_ref))
  
    # --- 计算微扰轨迹的 d(m_pert)/dt ---
    H_eff_pert = H_ext * e_x - H_deg * m_pert[2] * e_z
    T_field_pert = -gamma * np.cross(m_pert, H_eff_pert)
    T_stt_pert = gamma * a_j_t * np.cross(m_pert, np.cross(m_pert, e_x))
    T_total_pert = T_field_pert + T_stt_pert
    dmdt_pert = (1 / (1 + alpha**2)) * (T_total_pert + alpha * np.cross(m_pert, T_total_pert))
  
    return np.concatenate([dmdt_ref, dmdt_pert])


In [7]:

# --- 2. LLE 计算主函数 ---
def calculate_lle(get_aj_func, total_time=200, tau=0.5, d0=1e-8, transient_time=50):
    """
    计算给定输入信号下的最大李雅普诺夫指数。
    """
    # 初始化
    m_ref = np.random.rand(3) - 0.5
    m_ref /= np.linalg.norm(m_ref)
  
    # 首先让系统演化一段时间，摆脱初始瞬态效应
    if transient_time > 0:
        def single_llg(t, m):
            y_6d = llg_6d(t, np.concatenate([m,m]), get_aj_func)
            return y_6d[:3]
      
        sol_transient = solve_ivp(single_llg, [0, transient_time], m_ref, dense_output=True)
        m_ref = sol_transient.sol(transient_time)

    # 创建初始微扰轨迹
    pert_vec = np.random.rand(3) - 0.5
    pert_vec /= np.linalg.norm(pert_vec)
    m_pert = m_ref + d0 * pert_vec
  
    log_ratios = []
    current_time = 0
  
    num_steps = int(total_time / tau)
    for _ in range(num_steps):
        y0 = np.concatenate([m_ref, m_pert])
        t_span = [current_time, current_time + tau]
      
        # 演化一个时间步 tau
        sol = solve_ivp(lambda t, y: llg_6d(t, y, get_aj_func), t_span, y0, method='LSODA')
      
        # 提取演化后的状态
        m_ref_end = sol.y[:3, -1]
        m_pert_end = sol.y[3:, -1]
      
        # 计算分离距离
        d_final = np.linalg.norm(m_pert_end - m_ref_end)
      
        # 记录对数比
        if d_final > 0: # 避免 log(0)
            log_ratios.append(np.log(d_final / d0))
      
        # 重整化
        m_ref = m_ref_end
        m_pert = m_ref_end + d0 * (m_pert_end - m_ref_end) / d_final
      
        current_time += tau
      
    # 计算LLE
    if not log_ratios:
        return 0.0
  
    lle = np.mean(log_ratios) / tau
    return lle


In [9]:

# --- 3. 重现文献 Fig. 3(a) ---
if __name__ == "__main__":
    # 定义要扫描的 a_dc 值范围
    adc_values = np.linspace(100, 250, 31) # 从 100 Oe 到 250 Oe, 取31个点
    lle_results = []
  
    print("开始计算李雅普诺夫指数随 a_dc 的变化...")
    for a_dc in tqdm(adc_values):
        # 对于每个 a_dc，定义对应的 get_aj(t) 函数
        # 注意：这里我们只考虑直流输入，因为这是Fig 3(a)的情况
        def get_aj_constant_dc(t):
            return a_dc + a_ac * np.cos(omega_ac * t)

        # 计算LLE
        # 使用较短的 total_time 以便快速演示，要获得平滑曲线需要更长时间
        lle_val = calculate_lle(get_aj_constant_dc, total_time=200, tau=0.2, transient_time=100)
        lle_results.append(lle_val)
      
    print("计算完成。")
  
    # 绘制结果
    plt.figure(figsize=(10, 6))
    plt.plot(adc_values, lle_results, 'o-', label='Calculated LLE')
    plt.axhline(0, color='r', linestyle='--', label='$\lambda = 0$ (Chaos Threshold)')
    plt.title('Largest Lyapunov Exponent vs. DC Input $a_{dc}$', fontsize=14)
    plt.xlabel('$a_{dc}$ (Oe)', fontsize=12)
    plt.ylabel('Lyapunov Exponent $\lambda$ (ns$^{-1}$)', fontsize=12)
    plt.grid(True)
    plt.legend()
    plt.show()
    print("结果图已显示。")

<>:24: SyntaxWarning: invalid escape sequence '\l'
<>:27: SyntaxWarning: invalid escape sequence '\l'
<>:24: SyntaxWarning: invalid escape sequence '\l'
<>:27: SyntaxWarning: invalid escape sequence '\l'
/var/folders/7c/kwhn06r91h57xlqx9rqvvpmm0000gn/T/ipykernel_34697/1654982490.py:24: SyntaxWarning: invalid escape sequence '\l'
  plt.axhline(0, color='r', linestyle='--', label='$\lambda = 0$ (Chaos Threshold)')
/var/folders/7c/kwhn06r91h57xlqx9rqvvpmm0000gn/T/ipykernel_34697/1654982490.py:27: SyntaxWarning: invalid escape sequence '\l'
  plt.ylabel('Lyapunov Exponent $\lambda$ (ns$^{-1}$)', fontsize=12)


开始计算李雅普诺夫指数随 a_dc 的变化...


  3%|▎         | 1/31 [00:19<09:38, 19.28s/it]capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.



KeyboardInterrupt: 